# 全量微调

In [4]:
from transformers import AutoModel,AutoTokenizer,pipeline,Trainer,TrainingArguments,AutoModelForCausalLM,default_data_collator
from datasets import Dataset, load_dataset
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 1、加载模型和分词器

In [5]:
model_name="Qwen/Qwen3-0.6B"

In [6]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map='auto',
    torch_dtype="auto",
)
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

Loading weights: 100%|██████████| 311/311 [00:00<00:00, 498.31it/s, Materializing param=model.norm.weight]                              
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


## 2、加载训练数据

In [7]:
data=load_dataset('json',data_files='../../dataset/chinese_law_ft_dataset.json',split="train[:1000]")

In [8]:
dataset = data.train_test_split(
    train_size=0.7,
    shuffle=True,
    seed=7
)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'id'],
        num_rows: 700
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'id'],
        num_rows: 300
    })
})

## 3、数据预处理

In [10]:
def process_fun(example):
    content=[]
    for instruction,input,output in zip(example['instruction'],example['input'],example['output']):
        if input.strip():
            text=f'Human:{instruction}\n{input}\nAI:{output}'
            content.append(text)
        else:
            text=f'Human:{instruction}\nAI:{output}'
            content.append(text)
    
    encoded = tokenizer(
        content,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    labels = encoded["input_ids"].clone()
    for index,text in enumerate(content):
        answer_start=text.find('AI:')+len('AI:')
        question=text[:answer_start]
        question_ids=tokenizer.encode(question, add_special_tokens=False)
        question_length=len(question_ids)
        labels[index,:question_length]=-100
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": labels
    }


In [11]:
train_process_data=dataset['train'].map(process_fun,batched=True,remove_columns=dataset['train'].column_names)
test_process_data=dataset['test'].map(process_fun,batched=True,remove_columns=dataset['test'].column_names)

## 4、训练超参数配置

In [15]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="../../models/fine_tuning",
    logging_steps=100,
    logging_dir='./runs',
    eval_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    weight_decay=0.01,
    metric_for_best_model='eval_loss',
    gradient_accumulation_steps=4,  # 如果GPU内存有限
    bf16=True
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


## 5、训练模型

In [16]:
trainer=Trainer(
    model=model,
    args=training_args,
    eval_dataset=train_process_data,
    train_dataset=test_process_data,
    data_collator=default_data_collator,
)

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.396697
2,No log,0.380726
3,No log,0.395633


Writing model shards: 100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


TrainOutput(global_step=57, training_loss=0.6442729213781524, metrics={'train_runtime': 5483.2277, 'train_samples_per_second': 0.164, 'train_steps_per_second': 0.01, 'total_flos': 1647958818816000.0, 'train_loss': 0.6442729213781524, 'epoch': 3.0})

## 6、保存模型

In [18]:
trainer.save_model('../../models/train_qwen')

Writing model shards: 100%|██████████| 1/1 [00:01<00:00,  1.87s/it]
